In [3]:
!ls -a cell_images/

.  ..  Parasitized  Uninfected


In [15]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

Img_size=100
num_classes=2


Data_dir="cell_images"  #Directorry containg the images
Categories =["Uninfected","Parasitized"]  #two image categories

In [30]:
#Building the flatten cell image data set and Labels for colored images

Cell_image_Data=[]
Cell_image_Labels=[]

#Creating fuction to get Cell images data and labels to two lists
def create_Cell_image_data():
    for cat in Categories:
        path=os.path.join(Data_dir,cat) #path to Uninfected, Parasitized directories
        img_class=Categories.index(cat) #getting indexes of the two categories, 0-Uninfected and 1-Parasitized
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(Img_size,Img_size))#resizing images 
                new_array=new_array.flatten() #flatten the images
                Cell_image_Data.append(new_array)   #appending new_array data to Cell_image_Data
                Cell_image_Labels.append(img_class) #appending img_class and img_class to Cell_image_Labels
            except Exception as e:
                pass

create_Cell_image_data()          



In [31]:
#Converting both lists to numpy arrays
Cell_image_Data=np.array(Cell_image_Data)
Cell_image_Labels=np.array(Cell_image_Labels)


In [32]:
print(np.shape(Cell_image_Data))

(27558, 30000)


In [33]:
#Shuffleing data randomly
sf=np.arange(Cell_image_Data.shape[0])
np.random.shuffle(sf)
Cell_image_Data=Cell_image_Data[sf]
Cell_image_Labels=Cell_image_Labels[sf]

In [34]:
#Splitting data as 80% of Trainign set and 20% of Test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(Cell_image_Data,Cell_image_Labels,test_size=0.2,random_state=50)

In [35]:
#reshaping features data to match keras acceptable input forms

X_train=X_train.astype('float32')/255
X_test=X_test.astype('float32')/255



In [36]:
#one hot encording the Y variable
import keras
Y_train=keras.utils.to_categorical(Y_train,num_classes)
Y_test=keras.utils.to_categorical(Y_test,num_classes)

In [37]:
print(np.shape(X_train))
print(np.shape(Y_train))
print(np.shape(X_test))
print(np.shape(Y_test))

(22046, 30000)
(22046, 2)
(5512, 30000)
(5512, 2)


In [38]:
#Saving numpy arrays to disk to run CCN models later time
np.save('X_train_color_FL',X_train)
np.save('X_test_color_FL',X_test)
np.save('Y_train_color_FL',Y_train)
np.save('Y_test_color_FL',Y_test)

In [2]:
import numpy as np

X_train = np.load('X_train_color_FL.npy')
Y_train = np.load('Y_train_color_FL.npy')
X_test = np.load('X_test_color_FL.npy')
Y_test = np.load('Y_test_color_FL.npy')

In [3]:
print(np.shape(X_train))
print(np.shape(Y_train))
print(np.shape(X_test))
print(np.shape(Y_test))

(22046, 30000)
(22046, 2)
(5512, 30000)
(5512, 2)


In [ ]:
print(np.shape(X_train))
print(np.shape(X_test))


In [5]:
#Building the basic Neural Network
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

image_size=100
num_classes=2

model=Sequential()

model.add(Dense(units=64, activation='relu', input_shape=(image_size*image_size*3,)))
model.add(Dropout(0.1))

model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(num_classes, activation='sigmoid'))




Using TensorFlow backend.
/opt/ibm/conda/miniconda3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/ibm/conda/miniconda3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/ibm/conda/miniconda3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/ibm/conda/miniconda3.6/lib/python3.6

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                1920064   
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 1,922,210
Trainable params: 1,922,210
Non-trainable params: 0
_________________________________________________________________


In [7]:

model.compile(loss='binary_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train,
          batch_size=64,
          epochs=20,
          verbose=1,
          validation_split=0.20)
score=model.evaluate(X_test,Y_test,verbose=0)
print('Test loss:', score[0])
print('Test Accuracy', score[1])

Instructions for updating:
Use tf.cast instead.


Train on 17636 samples, validate on 4410 samples
Epoch 1/20
17636/17636 [==============================] - 13s 717us/step - loss: 0.7629 - acc: 0.4985 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/20
17636/17636 [==============================] - 11s 624us/step - loss: 0.6932 - acc: 0.4963 - val_loss: 0.6932 - val_acc: 0.4946
Epoch 3/20
17636/17636 [==============================] - 11s 634us/step - loss: 0.6932 - acc: 0.4980 - val_loss: 0.6932 - val_acc: 0.4946
Epoch 4/20
17636/17636 [==============================] - 11s 640us/step - loss: 0.6932 - acc: 0.4994 - val_loss: 0.6932 - val_acc: 0.4946
Epoch 5/20
17636/17636 [==============================] - 11s 627us/step - loss: 0.6932 - acc: 0.5005 - val_loss: 0.6932 - val_acc: 0.4946
Epoch 6/20
17636/17636 [==============================] - 11s 613us/step - loss: 0.6932 - acc: 0.4985 - val_loss: 0.6932 - val_acc: 0.4946
Epoch 7/20
17636/17636 [==============================] - 12s 658us/step - loss: 0.6932 - acc: 0.4953 - val_loss: 0.6